In [1]:
!wget --header="Host: datahack-prod.s3.amazonaws.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://datahack.analyticsvidhya.com/contest/janatahack-demand-forecasting/True/" "https://datahack-prod.s3.amazonaws.com/train_file/train_0irEZ2H.csv" -c -O 'train.csv'
!wget --header="Host: datahack-prod.s3.amazonaws.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://datahack.analyticsvidhya.com/contest/janatahack-demand-forecasting/True/" "https://datahack-prod.s3.amazonaws.com/test_file/test_nfaJ3J5.csv" -c -O 'test.csv'

--2020-07-12 15:49:17--  https://datahack-prod.s3.amazonaws.com/train_file/train_0irEZ2H.csv
Resolving datahack-prod.s3.amazonaws.com (datahack-prod.s3.amazonaws.com)... 52.219.66.36
Connecting to datahack-prod.s3.amazonaws.com (datahack-prod.s3.amazonaws.com)|52.219.66.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7822684 (7.5M) [text/csv]
Saving to: ‘train.csv’

train.csv           100%[===================>]   7.46M  3.38MB/s    in 2.2s    

2020-07-12 15:49:20 (3.38 MB/s) - ‘train.csv’ saved [7822684/7822684]

--2020-07-12 15:49:23--  https://datahack-prod.s3.amazonaws.com/test_file/test_nfaJ3J5.csv
Resolving datahack-prod.s3.amazonaws.com (datahack-prod.s3.amazonaws.com)... 52.219.64.124
Connecting to datahack-prod.s3.amazonaws.com (datahack-prod.s3.amazonaws.com)|52.219.64.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 684259 (668K) [text/csv]
Saving to: ‘test.csv’

test.csv            100%[===================>] 668.22K

In [2]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [73]:
def get_(total_price,base_price):
  if total_price > base_price:
    return 2
  if base_price > total_price:
    return 0
  return 1

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
merge = pd.concat([train,test])
scaler = MinMaxScaler()

print(merge.shape)

merge['total_price'] = merge['total_price'].transform(lambda x: x.fillna(x.mean()))
merge['loss_or_profit_percent'] = merge.apply(lambda x:abs(x['total_price'] - x['base_price'])/100,axis=1)
merge['loss_or_profit'] = merge.apply(lambda x:get_(x['total_price'],x['base_price']),axis=1)

for i in range(1,6,2):
  prices = merge.groupby(['store_id',"sku_id"])['total_price','base_price'].shift(i).rename(columns={'total_price':f'shift_{i}_total_price',
                                                                                                    'base_price':f'shift_{i}_base_price'})
  prices['record_ID'] = merge.record_ID
  merge = pd.merge(merge,prices,on="record_ID")

# # merge.week = pd.to_datetime(merge.week)
# # merge['Year'] = merge.week.dt.year
# # merge['Month'] = merge.week.dt.month
# # merge['Day'] = merge.week.dt.day
# # merge['Dayofweek'] = merge.week.dt.dayofweek
# # merge['DayOfyear'] = merge.week.dt.dayofyear
# # merge['Week'] = merge.week.dt.week
# # merge['Quarter'] = merge.week.dt.quarter 
# # merge['Is_month_start'] = merge.week.dt.is_month_start
# # merge['Is_month_end'] = merge.week.dt.is_month_end
# # merge['Is_quarter_start'] = merge.week.dt.is_quarter_start
# # merge['Is_quarter_end'] = merge.week.dt.is_quarter_end
# # merge['Is_year_start'] = merge.week.dt.is_year_start
# # merge['Is_year_end'] = merge.week.dt.is_year_end
# # merge['Semester'] = np.where(merge['Quarter'].isin([1,2]),1,2)
# # merge['Is_weekend'] = np.where(merge['Dayofweek'].isin([5,6]),1,0)
# # merge['Is_weekday'] = np.where(merge['Dayofweek'].isin([0,1,2,3,4]),1,0)
# m = merge.groupby(['store_id',"sku_id"])['total_price'].agg(func={'mean',"max","min","sum"}).reset_index().rename(columns={"mean":"total_mean",
#                                                                                                                        "max":"total_max",
#                                                                                                                        "min":"total_min",
#                                                                                                                        "sum":"total_sum"})
# merge = merge.merge(m,on=['store_id',"sku_id"])
# m = merge.groupby(['store_id',"sku_id"])['base_price'].agg(func={'mean',"max","min","sum"}).reset_index().rename(columns={"mean":"base_mean",
#                                                                                                                        "max":"base_max",
#                                                                                                                        "min":"base_min",
#                                                                                                                        "sum":"base_sum"})
# merge = merge.merge(m,on=['store_id',"sku_id"])



from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
encoders = defaultdict(LabelEncoder)

train = merge[~merge.units_sold.isna()]
train.dropna(inplace=True)

test = merge[merge.units_sold.isna()]

(164010, 9)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [74]:
print(train.shape)
train.head()

(144375, 17)


,record_ID,week,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold,loss_or_profit_percent,loss_or_profit,shift_1_total_price,shift_1_base_price,shift_3_total_price,shift_3_base_price,shift_5_total_price,shift_5_base_price
5775,8191,21/02/11,8091,216418,106.1625,106.1625,0,0,29.0,0.00000,1,106.8750,106.8750,96.9000,96.9000,99.0375,111.8625
5776,8192,21/02/11,8091,216419,106.8750,106.8750,0,0,18.0,0.00000,1,106.8750,106.8750,98.3250,98.3250,99.0375,99.0375
5777,8193,21/02/11,8091,216425,114.7125,133.2375,0,0,12.0,0.18525,0,133.2375,133.2375,133.2375,133.2375,133.9500,133.9500
5778,8194,21/02/11,8091,216233,116.8500,133.9500,0,0,33.0,0.17100,0,133.9500,133.9500,134.6625,134.6625,133.9500,133.9500
5779,8195,21/02/11,8091,217390,170.2875,170.2875,0,0,21.0,0.00000,1,140.3625,140.3625,143.2125,143.2125,141.0750,141.0750


In [71]:
train.nunique()

record_ID                 145530
week                         126
store_id                      76
sku_id                        28
total_price                  646
base_price                   569
is_featured_sku                2
is_display_sku                 2
units_sold                   708
loss_or_profit_percent       769
loss_or_profit                 3
shift_1_total_price          647
shift_1_base_price           571
shift_2_total_price          647
shift_2_base_price           571
shift_3_total_price          644
shift_3_base_price           569
shift_4_total_price          643
shift_4_base_price           567
total_min                    212
total_sum                   1146
total_mean                  1145
total_max                     93
base_min                     258
base_sum                    1133
base_mean                   1133
base_max                     102
dtype: int64

In [48]:
# from sklearn.ensemble import ExtraTreesRegressor
# from sklearn.model_selection import train_test_split

# X = train.drop(['units_sold',"record_ID","week"],axis=1)
# Y = train[['units_sold']].values

# x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2)

# model = ExtraTreesRegressor()
# model.fit(x_train,y_train)

# k = list(zip(x_train.columns,model.feature_importances_))
# pd.DataFrame(k,columns=['feature',"importance"]).sort_values("importance")

In [49]:
# use_cols = ['']

In [50]:
!pip install catboost

In [72]:
from sklearn.model_selection import train_test_split ,KFold
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_log_error
import math
from sklearn.linear_model import LinearRegression

folds = KFold(n_splits=5)

X = train.drop(['units_sold',"record_ID","week"],axis=1)
Y = train[['units_sold']]

params = {
    # "one_hot_max_size":75,
    "od_type":"Iter",
    "od_wait":25,
    "learning_rate":0.004,
    "eval_metric":"MSLE",
    "depth":5,
    "verbose":50,
    "boosting_type":"Plain"
}

scores = []
min_score = np.inf
for (train_idx,test_idx),i in zip(folds.split(X,Y),range(5)):
  
  print("Working on Fold ",i)

  test_set = (X.iloc[test_idx],Y.iloc[test_idx])
  model = CatBoostRegressor(**params)
  model.fit(X.iloc[train_idx],Y.iloc[train_idx],
            cat_features = ['store_id','sku_id',"is_featured_sku",'is_display_sku',"loss_or_profit"],
            eval_set = (test_set[0],test_set[1]))
  predictions = model.predict(test_set[0])
  predictions = [max(i,0) for i in predictions]
  # predictions = units_scaler.inverse_transform([predictions])
  score = math.sqrt(mean_squared_log_error(test_set[1],predictions))
  scores.append(score)
  if score < min_score:
    best_idx = train_idx 

  print("LB Score : ",100*score)
  print("-"*55)
  break

Working on Fold  0
0:	learn: 0.8613844	test: 0.9516495	best: 0.9516495 (0)	total: 104ms	remaining: 1m 43s
50:	learn: 0.7362291	test: 0.8434824	best: 0.8434824 (50)	total: 3.08s	remaining: 57.3s
100:	learn: 0.6513490	test: 0.7668991	best: 0.7668991 (100)	total: 6.07s	remaining: 54s
150:	learn: 0.5869536	test: 0.7080718	best: 0.7080718 (150)	total: 9.08s	remaining: 51s
200:	learn: 0.5379338	test: 0.6635801	best: 0.6635801 (200)	total: 12.3s	remaining: 48.8s
250:	learn: 0.4999104	test: 0.6313011	best: 0.6313011 (250)	total: 15.6s	remaining: 46.6s
300:	learn: 0.4695206	test: 0.6056238	best: 0.6056238 (300)	total: 18.7s	remaining: 43.4s
350:	learn: 0.4362827	test: 0.5879687	best: 0.5877386 (348)	total: 21.8s	remaining: 40.3s
Stopped by overfitting detector  (25 iterations wait)

bestTest = 0.584705577
bestIteration = 372

Shrink model to first 373 iterations.
LB Score :  76.46604368312926
-------------------------------------------------------


In [67]:
print(np.array(scores).mean())
val_idx = train[~train.index.isin(best_idx)].index.values

0.7693444775684238


In [41]:
test_set = (X.loc[val_idx],Y.loc[val_idx] )
model = CatBoostRegressor(**params)
model.fit(X.iloc[best_idx],Y.iloc[best_idx],
          cat_features = ['store_id','sku_id',"is_featured_sku",'is_display_sku',"loss_or_profit"],
          eval_set = (test_set[0],test_set[1]))

predictions = model.predict(test_set[0])
predictions = [max(i,0) for i in predictions]
# print((test_set[0]))
score = math.sqrt(mean_squared_log_error(test_set[1],predictions))

print(100*score)

Learning rate set to 0.027056
0:	learn: 0.8642978	test: 0.8509348	best: 0.8509348 (0)	total: 94.8ms	remaining: 15m 47s
50:	learn: 0.4216873	test: 0.4008464	best: 0.4008464 (50)	total: 3.41s	remaining: 11m 5s
100:	learn: 0.2860957	test: 0.2744163	best: 0.2744163 (100)	total: 6.92s	remaining: 11m 18s
150:	learn: 0.2538334	test: 0.2488527	best: 0.2488527 (150)	total: 10.8s	remaining: 11m 45s
200:	learn: 0.2391936	test: 0.2385183	best: 0.2385183 (200)	total: 14.8s	remaining: 11m 59s
250:	learn: 0.2297589	test: 0.2316104	best: 0.2316104 (250)	total: 18.8s	remaining: 12m 11s
300:	learn: 0.2224015	test: 0.2259792	best: 0.2259792 (300)	total: 22.9s	remaining: 12m 16s
350:	learn: 0.2169270	test: 0.2209519	best: 0.2209469 (349)	total: 26.6s	remaining: 12m 11s
400:	learn: 0.2136384	test: 0.2190745	best: 0.2190745 (400)	total: 30.6s	remaining: 12m 12s
450:	learn: 0.2106970	test: 0.2171026	best: 0.2171026 (450)	total: 34.5s	remaining: 12m 10s
Stopped by overfitting detector  (25 iterations wait)

b

In [42]:
if 'units_sold' in test.columns:
  test.drop(['units_sold'],axis=1,inplace=True)
test

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,record_ID,week,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,loss_or_profit_percent,loss_or_profit,shift_1_total_price,shift_1_base_price,shift_2_total_price,shift_2_base_price
150150,212645,16/07/13,8091,216418,108.3000,108.3000,0,0,0.000000,1,110.4375,110.4375,109.0125,109.0125
150151,212646,16/07/13,8091,216419,109.0125,109.0125,0,0,0.000000,1,108.3000,108.3000,109.0125,109.0125
150152,212647,16/07/13,8091,216425,133.9500,133.9500,0,0,0.000000,1,130.3875,130.3875,131.1000,131.1000
150153,212648,16/07/13,8091,216233,133.9500,133.9500,0,0,0.000000,1,134.6625,134.6625,134.6625,134.6625
150154,212649,16/07/13,8091,217390,176.7000,176.7000,0,0,0.000000,1,173.1375,173.1375,175.2750,175.2750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164005,232281,01/10/13,9984,223245,241.5375,241.5375,0,0,0.000000,1,241.5375,241.5375,241.5375,241.5375
164006,232282,01/10/13,9984,223153,240.8250,240.8250,0,0,0.000000,1,240.8250,240.8250,240.8250,240.8250
164007,232285,01/10/13,9984,245338,382.6125,401.8500,1,1,0.192375,0,378.3375,401.8500,401.8500,401.8500
164008,232286,01/10/13,9984,547934,191.6625,191.6625,0,0,0.000000,1,191.6625,191.6625,191.6625,191.6625


In [43]:
X = test.drop(["record_ID","week"],axis=1)
predictions = model.predict(X)
# predictions = units_scaler.inverse_transform([predictions])[0]
predictions = [max(i,0) for i in predictions]
test['units_sold'] = predictions
submission = test[['record_ID','units_sold']]
submission.to_csv("submission.csv",index=False)
submission

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,record_ID,units_sold
150150,212645,24.954103
150151,212646,25.317605
150152,212647,19.985441
150153,212648,24.158171
150154,212649,25.951765
...,...,...
164005,232281,31.811707
164006,232282,28.150876
164007,232285,33.090948
164008,232286,16.005940
